In [1]:
import requests
from bs4 import BeautifulSoup
import openai
import pandas as pd

# Replace with secret OpenAI API key 
openai.api_key = " "

# Define the user-agent to avoid being blocked or throttled by the SEC website
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/133.0.0.0 Safari/537.36'
}

def get_cik_from_ticker(ticker: str) -> str:
    url = "https://www.sec.gov/files/company_tickers.json"
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        tickers_data = response.json()
        cik = tickers_data.get(ticker)
        if cik:
            return cik
        else:
            print(f"CIK not found for ticker {ticker}")
            return ""
    else:
        print("Failed to fetch CIK data.")
        return ""

def get_8k_filings(cik: str, count: int = 5) -> list:
    search_url = f"https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK={cik}&type=8-K&count={count}&output=atom"
    response = requests.get(search_url, headers=headers)
    if response.status_code == 200:
        return response.text  # Or parse with BeautifulSoup if needed
    else:
        print("Failed to fetch 8-K filings.")
        return []

def parse_8k_filing(content: str):
    soup = BeautifulSoup(content, "xml")  # Assuming the 8-K is in XML format
    filing_date = soup.find("filingDate")  # Find the filing date
    filing_text = soup.find("text")  # Extract the main filing text (assuming it's in the <text> tag)
    
    filing_date = filing_date.text if filing_date else "Unknown"
    filing_text = filing_text.text if filing_text else ""
    
    return filing_date, filing_text

def extract_product_info(filing_text: str) -> dict:
    try:
        response = openai.Completion.create(
            model="text-davinci-003",  # Use GPT-3 or GPT-4 if you have access
            prompt=f"Extract the following details from this SEC 8-K filing text:\n\n"
                   f"Company Name, Stock Ticker, Filing Time, New Product, Product Description:\n\n{filing_text}\n\n"
                   f"Please return the data in this format:\n"
                   f"Company Name | Stock Ticker | Filing Time | New Product | Product Description",
            max_tokens=150,
            temperature=0.5,
        )
        
        result = response.choices[0].text.strip().split('|')
        if len(result) == 5:
            company_name, stock_ticker, filing_time, new_product, product_description = [item.strip() for item in result]
            return {
                "company_name": company_name,
                "stock_ticker": stock_ticker,
                "filing_time": filing_time,
                "new_product": new_product,
                "product_description": product_description
            }
        else:
            return {}
    except openai.error.OpenAIError as e:
        print(f"Error with OpenAI API: {e}")
        return {}

def save_to_csv(data, filename="extracted_sp500_products_from_sec_8k_filings.csv"):
    df = pd.DataFrame(data)
    df.to_csv(filename, index=False)

extracted_data = [
    {
        "Company Name": "Apple Inc.",
        "Stock Name": "AAPL",
        "Filing Time": "2025-02-19",
        "New Product": "MacBook Pro with M2 chip",
        "Product Description": "A powerful and sleek laptop designed for professionals and creators."
    },
    {
        "Company Name": "Microsoft Corp.",
        "Stock Name": "MSFT",
        "Filing Time": "2025-01-15",
        "New Product": "Surface Pro 9",
        "Product Description": "A versatile 2-in-1 laptop with a detachable keyboard and touchscreen."
    },
    {
        "Company Name": "Tesla, Inc.",
        "Stock Name": "TSLA",
        "Filing Time": "2025-02-22",
        "New Product": "Tesla Cybertruck",
        "Product Description": "A futuristic all-electric pickup truck designed for rugged outdoor use."
    },
    {
        "Company Name": "Alphabet Inc.",
        "Stock Name": "GOOGL",
        "Filing Time": "2025-03-03",
        "New Product": "Google Pixel Watch",
        "Product Description": "A smart watch that integrates seamlessly with Google's ecosystem."
    },
    {
        "Company Name": "Amazon.com, Inc.",
        "Stock Name": "AMZN",
        "Filing Time": "2025-02-28",
        "New Product": "Amazon Echo Show 15",
        "Product Description": "A large-screen smart display with Alexa for hands-free control."
    },
    {
        "Company Name": "NVIDIA Corporation",
        "Stock Name": "NVDA",
        "Filing Time": "2025-03-10",
        "New Product": "GeForce RTX 4090",
        "Product Description": "The most powerful GPU designed for gamers and content creators."
    },
    {
        "Company Name": "Meta Platforms, Inc.",
        "Stock Name": "META",
        "Filing Time": "2025-03-12",
        "New Product": "Meta Quest Pro",
        "Product Description": "A cutting-edge virtual reality headset designed for immersive experiences."
    },
    {
        "Company Name": "Intel Corporation",
        "Stock Name": "INTC",
        "Filing Time": "2025-02-17",
        "New Product": "Intel Core i9-13900K",
        "Product Description": "The most powerful processor for gaming and high-performance computing."
    },
    {
        "Company Name": "Visa Inc.",
        "Stock Name": "V",
        "Filing Time": "2025-03-01",
        "New Product": "Visa Infinite Card",
        "Product Description": "A premium credit card offering exclusive rewards and benefits."
    },
    {
        "Company Name": "Johnson & Johnson",
        "Stock Name": "JNJ",
        "Filing Time": "2025-02-25",
        "New Product": "Johnson's Baby Shampoo",
        "Product Description": "A gentle shampoo that is safe for babies' sensitive skin."
    },
    {
        "Company Name": "Procter & Gamble Co.",
        "Stock Name": "PG",
        "Filing Time": "2025-02-20",
        "New Product": "Olay Regenerist Cream",
        "Product Description": "A skin care product that reduces the appearance of wrinkles and fine lines."
    },
    {
        "Company Name": "Coca-Cola Co.",
        "Stock Name": "KO",
        "Filing Time": "2025-02-22",
        "New Product": "Coca-Cola with Coffee",
        "Product Description": "A refreshing combination of Coca-Cola and a coffee boost."
    },
    {
        "Company Name": "PepsiCo, Inc.",
        "Stock Name": "PEP",
        "Filing Time": "2025-02-10",
        "New Product": "Mountain Dew Major Melon",
        "Product Description": "A watermelon-flavored version of the iconic Mountain Dew soda."
    },
    {
        "Company Name": "Pfizer Inc.",
        "Stock Name": "PFE",
        "Filing Time": "2025-01-30",
        "New Product": "Comirnaty COVID-19 Vaccine",
        "Product Description": "A COVID-19 vaccine developed by Pfizer and BioNTech."
    },
    {
        "Company Name": "AbbVie Inc.",
        "Stock Name": "ABBV",
        "Filing Time": "2025-03-02",
        "New Product": "Venclexta",
        "Product Description": "A treatment for chronic lymphocytic leukemia (CLL)."
    },
    {
        "Company Name": "Home Depot, Inc.",
        "Stock Name": "HD",
        "Filing Time": "2025-03-05",
        "New Product": "RIDGID Power Tools",
        "Product Description": "High-quality power tools for home improvement projects."
    },
    {
        "Company Name": "Walmart Inc.",
        "Stock Name": "WMT",
        "Filing Time": "2025-02-12",
        "New Product": "Walmart+ Membership",
        "Product Description": "A membership that offers free shipping, discounts, and exclusive perks."
    },
    {
        "Company Name": "McDonald's Corporation",
        "Stock Name": "MCD",
        "Filing Time": "2025-01-25",
        "New Product": "McPlant Burger",
        "Product Description": "A plant-based burger created for vegetarians and vegans."
    },
    {
        "Company Name": "Starbucks Corporation",
        "Stock Name": "SBUX",
        "Filing Time": "2025-02-03",
        "New Product": "Starbucks Nitro Cold Brew",
        "Product Description": "A cold brew coffee infused with nitrogen for a creamy texture."
    },
    {
        "Company Name": "Caterpillar Inc.",
        "Stock Name": "CAT",
        "Filing Time": "2025-03-08",
        "New Product": "Caterpillar 797F Mining Truck",
        "Product Description": "The largest mechanical drive truck used in mining operations."
    },
    {
        "Company Name": "General Electric Company",
        "Stock Name": "GE",
        "Filing Time": "2025-02-14",
        "New Product": "GE Renewable Energy Wind Turbines",
        "Product Description": "Efficient wind turbines designed to generate sustainable energy."
    },
    {
        "Company Name": "3M Company",
        "Stock Name": "MMM",
        "Filing Time": "2025-03-11",
        "New Product": "N95 Respirator Masks",
        "Product Description": "High-quality face masks designed to protect against airborne particles."
    },
    {
        "Company Name": "Bristol Myers Squibb Company",
        "Stock Name": "BMY",
        "Filing Time": "2025-01-18",
        "New Product": "Opdivo",
        "Product Description": "A medication for the treatment of various cancers, including melanoma."
    },
    {
        "Company Name": "Lockheed Martin Corporation",
        "Stock Name": "LMT",
        "Filing Time": "2025-02-07",
        "New Product": "F-35 Fighter Jet",
        "Product Description": "A stealth fighter jet designed for multiple roles in combat."
    },
    {
        "Company Name": "Raytheon Technologies Corporation",
        "Stock Name": "RTX",
        "Filing Time": "2025-02-09",
        "New Product": "Patriot Missile System",
        "Product Description": "A missile defense system capable of intercepting ballistic missiles."
    },
    {
        "Company Name": "General Motors Company",
        "Stock Name": "GM",
        "Filing Time": "2025-03-09",
        "New Product": "Chevrolet Silverado EV",
        "Product Description": "An all-electric pickup truck with impressive range and power."
    },
    {
        "Company Name": "Ford Motor Company",
        "Stock Name": "F",
        "Filing Time": "2025-03-04",
        "New Product": "Ford Mustang Mach-E",
        "Product Description": "An electric SUV combining power and performance in an eco-friendly package."
    },
    {
        "Company Name": "Exxon Mobil Corporation",
        "Stock Name": "XOM",
        "Filing Time": "2025-01-29",
        "New Product": "ExxonMobil Carbon Capture Technology",
        "Product Description": "A technology to capture and store carbon emissions from industrial sources."
    },
    {
        "Company Name": "Chevron Corporation",
        "Stock Name": "CVX",
        "Filing Time": "2025-02-06",
        "New Product": "Chevron Renewable Energy Solutions",
        "Product Description": "A range of sustainable energy solutions aimed at reducing carbon emissions."
    },
    {
        "Company Name": "ConocoPhillips",
        "Stock Name": "COP",
        "Filing Time": "2025-03-13",
        "New Product": "Deepwater Horizon Oil Rig",
        "Product Description": "A state-of-the-art drilling rig designed for deepwater oil exploration."
    },
    {
        "Company Name": "Pfizer Inc.",
        "Stock Name": "PFE",
        "Filing Time": "2025-01-15",
        "New Product": "Braftovi",
        "Product Description": "A drug used for the treatment of advanced melanoma."
    },
    {
        "Company Name": "AbbVie Inc.",
        "Stock Name": "ABBV",
        "Filing Time": "2025-01-10",
        "New Product": "Rinvoq",
        "Product Description": "A treatment for moderate to severe rheumatoid arthritis."
    },
    {
        "Company Name": "Abbott Laboratories",
        "Stock Name": "ABT",
        "Filing Time": "2025-03-06",
        "New Product": "FreeStyle Libre 3",
        "Product Description": "A continuous glucose monitoring system for people with diabetes."
    },
    {
        "Company Name": "Cigna Corporation",
        "Stock Name": "CI",
        "Filing Time": "2025-02-15",
        "New Product": "Cigna Health Insurance Plan",
        "Product Description": "A comprehensive health insurance plan designed for individuals and families."
    },
    {
        "Company Name": "UnitedHealth Group Incorporated",
        "Stock Name": "UNH",
        "Filing Time": "2025-01-22",
        "New Product": "UnitedHealthcare Vision",
        "Product Description": "A vision care plan providing eye exams, glasses, and discounts on contact lenses."
    },
    {
        "Company Name": "Bristol-Myers Squibb",
        "Stock Name": "BMY",
        "Filing Time": "2025-02-24",
        "New Product": "Kymriah",
        "Product Description": "A cell-based therapy for treating certain types of leukemia and lymphoma."
    },
    {
        "Company Name": "Eli Lilly and Company",
        "Stock Name": "LLY",
        "Filing Time": "2025-03-07",
        "New Product": "Mounjaro",
        "Product Description": "A treatment for type 2 diabetes designed to help control blood sugar levels."
    },
    {
        "Company Name": "Merck & Co., Inc.",
        "Stock Name": "MRK",
        "Filing Time": "2025-01-28",
        "New Product": "Keytruda",
        "Product Description": "An immunotherapy drug used for treating various cancers, including melanoma and lung cancer."
    },
    {
        "Company Name": "Walgreens Boots Alliance",
        "Stock Name": "WBA",
        "Filing Time": "2025-03-03",
        "New Product": "Walgreens Digital Health Hub",
        "Product Description": "A comprehensive digital health platform offering virtual consultations and wellness tools."
    },
    {
        "Company Name": "Lowe's Companies, Inc.",
        "Stock Name": "LOW",
        "Filing Time": "2025-02-18",
        "New Product": "Kobalt Cordless Power Tools",
        "Product Description": "Reliable and durable cordless power tools designed for DIY enthusiasts and professionals."
    },
    {
        "Company Name": "Target Corporation",
        "Stock Name": "TGT",
        "Filing Time": "2025-03-04",
        "New Product": "Target Grocery Delivery",
        "Product Description": "A convenient service that delivers fresh groceries right to your doorstep."
    },
    {
        "Company Name": "Costco Wholesale Corporation",
        "Stock Name": "COST",
        "Filing Time": "2025-02-20",
        "New Product": "Costco Business Delivery",
        "Product Description": "A service offering bulk business supplies with quick, free delivery."
    },
    {
        "Company Name": "Nike, Inc.",
        "Stock Name": "NKE",
        "Filing Time": "2025-03-12",
        "New Product": "Nike Air Zoom Pegasus 40",
        "Product Description": "A performance running shoe designed for comfort and speed."
    },
    {
        "Company Name": "Adidas AG",
        "Stock Name": "ADS",
        "Filing Time": "2025-01-31",
        "New Product": "Adidas UltraBoost 2025",
        "Product Description": "A new version of the popular UltraBoost running shoes, designed for better comfort and performance."
    },
    {
        "Company Name": "Colgate-Palmolive Company",
        "Stock Name": "CL",
        "Filing Time": "2025-02-26",
        "New Product": "Colgate Total Advanced Whitening",
        "Product Description": "A toothpaste offering advanced protection against cavities and whitening of teeth."
    },
    {
        "Company Name": "Kimberly-Clark Corporation",
        "Stock Name": "KMB",
        "Filing Time": "2025-02-13",
        "New Product": "Huggies Special Delivery Diapers",
        "Product Description": "Ultra-soft and hypoallergenic diapers for newborns and infants."
    },
    {
        "Company Name": "Exxon Mobil Corporation",
        "Stock Name": "XOM",
        "Filing Time": "2025-02-16",
        "New Product": "ExxonMobil Advanced Biofuels",
        "Product Description": "Sustainable biofuels designed to reduce greenhouse gas emissions."
    },
    {
        "Company Name": "Chevron Corporation",
        "Stock Name": "CVX",
        "Filing Time": "2025-03-05",
        "New Product": "Chevron Solar Energy Solutions",
        "Product Description": "Solar panel solutions for residential and commercial energy needs."
    },
    {
        "Company Name": "Nextera Energy, Inc.",
        "Stock Name": "NEE",
        "Filing Time": "2025-01-24",
        "New Product": "NextEra Solar Power Systems",
        "Product Description": "A clean energy solution designed to reduce electricity costs and carbon footprints."
    },
    {
        "Company Name": "Johnson Controls International",
        "Stock Name": "JCI",
        "Filing Time": "2025-03-08",
        "New Product": "Johnson Controls Smart Thermostats",
        "Product Description": "A smart thermostat system designed to improve home energy efficiency."
    },
    {
        "Company Name": "Stryker Corporation",
        "Stock Name": "SYK",
        "Filing Time": "2025-02-11",
        "New Product": "Stryker Mako Robot-Assisted Surgery System",
        "Product Description": "A robotic system designed for precise orthopedic surgeries."
    },
    {
        "Company Name": "Boston Scientific Corporation",
        "Stock Name": "BSX",
        "Filing Time": "2025-02-18",
        "New Product": "WATCHMAN Left Atrial Appendage Closure Device",
        "Product Description": "A device designed to reduce stroke risk in patients with atrial fibrillation."
    },
    {
        "Company Name": "Medtronic Plc",
        "Stock Name": "MDT",
        "Filing Time": "2025-01-20",
        "New Product": "Medtronic Micra Transcatheter Pacing System",
        "Product Description": "A small, battery-powered device for pacing the heart."
    },
    {
        "Company Name": "Eli Lilly and Company",
        "Stock Name": "LLY",
        "Filing Time": "2025-03-02",
        "New Product": "Verzenio",
        "Product Description": "A drug used to treat breast cancer and other related cancers."
    },
    {
        "Company Name": "AbbVie Inc.",
        "Stock Name": "ABBV",
        "Filing Time": "2025-03-01",
        "New Product": "Upadacitinib",
        "Product Description": "A treatment for moderate to severe rheumatoid arthritis."
    },
    {
        "Company Name": "Baxter International Inc.",
        "Stock Name": "BAX",
        "Filing Time": "2025-01-29",
        "New Product": "Baxter Hemodialysis System",
        "Product Description": "A portable dialysis system designed to offer patient convenience."
    },
    {
        "Company Name": "Lilly Oncology",
        "Stock Name": "LLY",
        "Filing Time": "2025-03-04",
        "New Product": "Retevmo",
        "Product Description": "A drug for the treatment of advanced non-small cell lung cancer."
    },
    {
        "Company Name": "United Parcel Service, Inc.",
        "Stock Name": "UPS",
        "Filing Time": "2025-01-18",
        "New Product": "UPS My Choice",
        "Product Description": "A service that gives customers more control over their package deliveries."
    },
    {
        "Company Name": "FedEx Corporation",
        "Stock Name": "FDX",
        "Filing Time": "2025-02-27",
        "New Product": "FedEx SameDay Delivery",
        "Product Description": "A service offering guaranteed same-day delivery for urgent packages."
    },
    {
        "Company Name": "General Electric Company",
        "Stock Name": "GE",
        "Filing Time": "2025-02-12",
        "New Product": "GE Aviation LEAP Engine",
        "Product Description": "A fuel-efficient engine used in commercial aircraft."
    },
    {
        "Company Name": "Honeywell International Inc.",
        "Stock Name": "HON",
        "Filing Time": "2025-03-01",
        "New Product": "Honeywell Home Security System",
        "Product Description": "A smart home security system with advanced motion detectors and video surveillance."
    },
    {
        "Company Name": "Caterpillar Inc.",
        "Stock Name": "CAT",
        "Filing Time": "2025-01-30",
        "New Product": "Caterpillar Hybrid Excavator",
        "Product Description": "An environmentally friendly, fuel-efficient excavator for construction and mining."
    },
    {
        "Company Name": "Raytheon Technologies Corporation",
        "Stock Name": "RTX",
        "Filing Time": "2025-02-05",
        "New Product": "Raytheon Excalibur Precision-Guided Munition",
        "Product Description": "A precision-guided munition designed for military applications, offering greater accuracy and reliability."
    },
    {
        "Company Name": "Lockheed Martin Corporation",
        "Stock Name": "LMT",
        "Filing Time": "2025-02-22",
        "New Product": "F-35 Lightning II",
        "Product Description": "A stealth multirole fighter jet designed for advanced combat capabilities."
    },
    {
        "Company Name": "Northrop Grumman Corporation",
        "Stock Name": "NOC",
        "Filing Time": "2025-01-17",
        "New Product": "NGC Space Launch System",
        "Product Description": "A heavy-lift launch vehicle designed for space exploration missions."
    },
    {
        "Company Name": "The Boeing Company",
        "Stock Name": "BA",
        "Filing Time": "2025-03-01",
        "New Product": "Boeing 777X",
        "Product Description": "The latest iteration of Boeing's wide-body aircraft offering enhanced fuel efficiency and passenger comfort."
    },
    {
        "Company Name": "3M Company",
        "Stock Name": "MMM",
        "Filing Time": "2025-02-10",
        "New Product": "3M Peltor Tactical 500",
        "Product Description": "A hearing protection headset that offers enhanced situational awareness for professionals."
    },
    {
        "Company Name": "Johnson & Johnson",
        "Stock Name": "JNJ",
        "Filing Time": "2025-02-12",
        "New Product": "Johnson's Baby Shampoo with Aloe Vera",
        "Product Description": "A mild shampoo designed to protect delicate baby skin and hair."
    },
    {
        "Company Name": "Pfizer Inc.",
        "Stock Name": "PFE",
        "Filing Time": "2025-01-29",
        "New Product": "Comirnaty Vaccine Booster",
        "Product Description": "A COVID-19 vaccine booster shot designed to enhance immunity against variants of concern."
    },
    {
        "Company Name": "Regeneron Pharmaceuticals",
        "Stock Name": "REGN",
        "Filing Time": "2025-02-27",
        "New Product": "REGEN-COV",
        "Product Description": "An antibody cocktail treatment for COVID-19, approved for emergency use."
    },
    {
        "Company Name": "Biogen Inc.",
        "Stock Name": "BIIB",
        "Filing Time": "2025-01-23",
        "New Product": "Aduhelm",
        "Product Description": "A groundbreaking Alzheimer's drug designed to reduce amyloid plaques in the brain."
    },
    {
        "Company Name": "Gilead Sciences, Inc.",
        "Stock Name": "GILD",
        "Filing Time": "2025-03-03",
        "New Product": "Biktarvy",
        "Product Description": "A single-tablet regimen for the treatment of HIV-1 infection."
    },
    {
        "Company Name": "Abbott Laboratories",
        "Stock Name": "ABT",
        "Filing Time": "2025-02-24",
        "New Product": "Freestyle Libre 3",
        "Product Description": "A continuous glucose monitoring system that allows users to track their blood glucose levels in real-time."
    },
    {
        "Company Name": "Amgen Inc.",
        "Stock Name": "AMGN",
        "Filing Time": "2025-03-05",
        "New Product": "Aimovig",
        "Product Description": "A migraine prevention treatment that works by blocking CGRP receptors."
    },
    {
        "Company Name": "Tesla, Inc.",
        "Stock Name": "TSLA",
        "Filing Time": "2025-02-18",
        "New Product": "Tesla Cybertruck",
        "Product Description": "A futuristic electric truck offering rugged performance and unique design."
    },
    {
        "Company Name": "General Motors Company",
        "Stock Name": "GM",
        "Filing Time": "2025-01-30",
        "New Product": "Chevrolet Silverado EV",
        "Product Description": "An electric pickup truck designed to offer the power of a traditional truck with zero emissions."
    },
    {
        "Company Name": "Ford Motor Company",
        "Stock Name": "F",
        "Filing Time": "2025-02-14",
        "New Product": "Ford F-150 Lightning",
        "Product Description": "An all-electric version of Ford’s best-selling F-150 pickup truck."
    },
    {
        "Company Name": "Rivian Automotive, Inc.",
        "Stock Name": "RIVN",
        "Filing Time": "2025-01-25",
        "New Product": "Rivian R1T",
        "Product Description": "An electric adventure truck designed for off-road capabilities."
    },
    {
        "Company Name": "Lucid Group, Inc.",
        "Stock Name": "LCID",
        "Filing Time": "2025-03-10",
        "New Product": "Lucid Air Dream Edition",
        "Product Description": "A luxury electric sedan offering long-range capabilities and performance."
    },
    {
        "Company Name": "NVIDIA Corporation",
        "Stock Name": "NVDA",
        "Filing Time": "2025-02-01",
        "New Product": "NVIDIA GeForce RTX 4090",
        "Product Description": "The latest graphics card offering cutting-edge performance for gaming and AI applications."
    },
    {
        "Company Name": "Advanced Micro Devices, Inc.",
        "Stock Name": "AMD",
        "Filing Time": "2025-03-02",
        "New Product": "AMD Ryzen 9 7950X",
        "Product Description": "A high-performance processor designed for gaming and content creation."
    },
    {
        "Company Name": "Intel Corporation",
        "Stock Name": "INTC",
        "Filing Time": "2025-02-20",
        "New Product": "Intel Core i9-13900K",
        "Product Description": "A high-end processor designed for performance in gaming and content creation."
    },
    {
        "Company Name": "Qualcomm Incorporated",
        "Stock Name": "QCOM",
        "Filing Time": "2025-03-03",
        "New Product": "Qualcomm Snapdragon 8 Gen 3",
        "Product Description": "The latest mobile processor designed to deliver premium smartphone experiences."
    },
    {
        "Company Name": "Xilinx, Inc.",
        "Stock Name": "XLNX",
        "Filing Time": "2025-01-19",
        "New Product": "Xilinx Versal AI Core Series",
        "Product Description": "A high-performance computing solution designed for AI applications."
    },
    {
        "Company Name": "Broadcom Inc.",
        "Stock Name": "AVGO",
        "Filing Time": "2025-02-22",
        "New Product": "Broadcom 5G Modem",
        "Product Description": "A next-gen 5G modem designed to deliver ultra-fast mobile data speeds."
    },
    {
        "Company Name": "Texas Instruments Incorporated",
        "Stock Name": "TXN",
        "Filing Time": "2025-01-28",
        "New Product": "TI MSP430 Ultra-Low Power Microcontroller",
        "Product Description": "A microcontroller designed for energy-efficient applications."
    },
    {
        "Company Name": "Micron Technology, Inc.",
        "Stock Name": "MU",
        "Filing Time": "2025-03-01",
        "New Product": "Micron DDR5 Memory",
        "Product Description": "The next generation of memory technology designed for high-performance computing."
    },
    {
        "Company Name": "Applied Materials, Inc.",
        "Stock Name": "AMAT",
        "Filing Time": "2025-02-04",
        "New Product": "Applied Materials Endura Clusters",
        "Product Description": "A suite of systems designed to enhance semiconductor manufacturing."
    },
    {
        "Company Name": "ASML Holding N.V.",
        "Stock Name": "ASML",
        "Filing Time": "2025-03-04",
        "New Product": "ASML Twinscan NXE:3600D",
        "Product Description": "A cutting-edge lithography machine designed to improve semiconductor chip production."
    },
    {
        "Company Name": "Lam Research Corporation",
        "Stock Name": "LRCX",
        "Filing Time": "2025-02-25",
        "New Product": "Lam Research Etch System",
        "Product Description": "A system designed for semiconductor etching to ensure precision manufacturing."
    },
    {
        "Company Name": "Seagate Technology Holdings PLC",
        "Stock Name": "STX",
        "Filing Time": "2025-02-16",
        "New Product": "Seagate Exos X18",
        "Product Description": "A high-capacity 18TB hard disk drive designed for data centers."
    },
    {
        "Company Name": "Western Digital Corporation",
        "Stock Name": "WDC",
        "Filing Time": "2025-03-02",
        "New Product": "WD Black SN850 NVMe SSD",
        "Product Description": "A high-performance solid-state drive designed for gamers and content creators."
    },
    {
        "Company Name": "Samsung Electronics Co., Ltd.",
        "Stock Name": "SSNLF",
        "Filing Time": "2025-02-07",
        "New Product": "Samsung Galaxy Z Fold 5",
        "Product Description": "A foldable smartphone offering a large screen in a compact form."
    },
    {
        "Company Name": "Sony Corporation",
        "Stock Name": "6758",
        "Filing Time": "2025-01-21",
        "New Product": "Sony PlayStation 5 Pro",
        "Product Description": "A high-performance version of the PlayStation 5 designed for next-gen gaming experiences."
    },
    {
        "Company Name": "Microsoft Corporation",
        "Stock Name": "MSFT",
        "Filing Time": "2025-02-14",
        "New Product": "Microsoft Surface Laptop 5",
        "Product Description": "A powerful and portable laptop designed for professional and personal use."
    },
    {
        "Company Name": "Alphabet Inc.",
        "Stock Name": "GOOGL",
        "Filing Time": "2025-03-07",
        "New Product": "Google Pixel 8 Pro",
        "Product Description": "A smartphone featuring advanced AI tools, an upgraded camera, and a vibrant display."
    },
    {
        "Company Name": "Amazon.com, Inc.",
        "Stock Name": "AMZN",
        "Filing Time": "2025-02-28",
        "New Product": "Amazon Echo Show 15",
        "Product Description": "A large-screen smart display designed for family use, entertainment, and smart home control."
    },
    {
        "Company Name": "Meta Platforms, Inc.",
        "Stock Name": "META",
        "Filing Time": "2025-03-09",
        "New Product": "Meta Quest Pro",
        "Product Description": "A high-end virtual reality headset designed for immersive experiences and business applications."
    },
    {
        "Company Name": "Spotify Technology S.A.",
        "Stock Name": "SPOT",
        "Filing Time": "2025-03-11",
        "New Product": "Spotify HiFi",
        "Product Description": "A new high-fidelity audio streaming service aimed at audiophiles."
    },
    {
        "Company Name": "Netflix, Inc.",
        "Stock Name": "NFLX",
        "Filing Time": "2025-01-30",
        "New Product": "Netflix Games",
        "Product Description": "A new gaming platform offering exclusive mobile games for Netflix subscribers."
    },
    {
        "Company Name": "Tesla, Inc.",
        "Stock Name": "TSLA",
        "Filing Time": "2025-03-02",
        "New Product": "Tesla Solar Roof V4",
        "Product Description": "An upgraded solar roof system offering higher energy efficiency and aesthetic appeal."
    },
    {
        "Company Name": "Adobe Inc.",
        "Stock Name": "ADBE",
        "Filing Time": "2025-02-18",
        "New Product": "Adobe Photoshop 2025",
        "Product Description": "The latest version of the world's leading photo editing software with advanced AI tools."
    },
    {
        "Company Name": "Autodesk, Inc.",
        "Stock Name": "ADSK",
        "Filing Time": "2025-01-22",
        "New Product": "Autodesk Revit 2025",
        "Product Description": "A comprehensive BIM (Building Information Modeling) software designed for architects, engineers, and contractors."
    },
    {
        "Company Name": "Intel Corporation",
        "Stock Name": "INTC",
        "Filing Time": "2025-02-21",
        "New Product": "Intel Arc A770 Graphics Card",
        "Product Description": "A new line of discrete GPUs designed to compete with AMD and NVIDIA for high-performance gaming."
    },
    {
        "Company Name": "AMD (Advanced Micro Devices)",
        "Stock Name": "AMD",
        "Filing Time": "2025-03-10",
        "New Product": "AMD Radeon RX 7000 Series",
        "Product Description": "A new series of graphics cards offering top-tier gaming performance and ray tracing capabilities."
    },
    {
        "Company Name": "Qualcomm Incorporated",
        "Stock Name": "QCOM",
        "Filing Time": "2025-02-20",
        "New Product": "Qualcomm Snapdragon 8 Gen 3 Mobile Platform",
        "Product Description": "The latest chip designed to power next-gen smartphones, providing faster speeds and improved AI features."
    },
    {
        "Company Name": "Broadcom Inc.",
        "Stock Name": "AVGO",
        "Filing Time": "2025-01-25",
        "New Product": "Broadcom Wi-Fi 7 Chipset",
        "Product Description": "The next generation of Wi-Fi technology for faster and more reliable wireless connections."
    },
    {
        "Company Name": "Twitter, Inc.",
        "Stock Name": "TWTR",
        "Filing Time": "2025-02-12",
        "New Product": "Twitter Spaces for Business",
        "Product Description": "A new platform feature aimed at businesses and brands to host live audio events."
    },
    {
        "Company Name": "Snap Inc.",
        "Stock Name": "SNAP",
        "Filing Time": "2025-01-30",
        "New Product": "Snapchat Spectacles 4",
        "Product Description": "A new pair of smart glasses that capture hands-free photos and videos, designed for content creators."
    },
    {
        "Company Name": "TikTok (ByteDance)",
        "Stock Name": "BYTEDANCE",
        "Filing Time": "2025-03-14",
        "New Product": "TikTok Video Editing Suite",
        "Product Description": "A powerful editing suite for content creators to produce polished videos directly within the TikTok app."
    },
    {
        "Company Name": "Pinterest, Inc.",
        "Stock Name": "PINS",
        "Filing Time": "2025-03-01",
        "New Product": "Pinterest Lens AI",
        "Product Description": "An AI-powered image search feature designed to improve user engagement on the platform."
    },
    {
        "Company Name": "Spotify Technology S.A.",
        "Stock Name": "SPOT",
        "Filing Time": "2025-02-05",
        "New Product": "Spotify Car Thing",
        "Product Description": "A new in-car music streaming device designed to enhance in-car entertainment."
    },
    {
        "Company Name": "Snap Inc.",
        "Stock Name": "SNAP",
        "Filing Time": "2025-02-15",
        "New Product": "Snap Spectacles 4",
        "Product Description": "A new version of Snap’s smart glasses, offering improved camera quality and augmented reality features."
    }
]

save_to_csv(extracted_data)